## vgg块

In [18]:
import torchvision
import torch.nn as nn
import torch

def vgg_block(in_channels, out_channels, num_conv_layers):
    layers = []
    for _ in range(num_conv_layers):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU(inplace=True))
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

class VGG11(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.conv_layers = nn.Sequential(
            vgg_block(in_channels, 64, 1),      # conv1
            vgg_block(64, 128, 1),            # conv2
            vgg_block(128, 256, 2),           # conv3
            vgg_block(256, 512, 2),           # conv4
            vgg_block(512, 512, 2)            # conv5
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )
        self.softmax = nn.Softmax()
    def forward(self, x):
        x = x.view(batch_sizes,1,28,28)
        trans =[]
        trans.append(torchvision.transforms.Resize(224))
        transform = torchvision.transforms.Compose(trans) 
        x = transform(x[:128]) 
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        
        x = self.softmax(x)
       
        return x


In [19]:
import torchvision

trans =[]
trans.append(torchvision.transforms.Resize(224))

transform = torchvision.transforms.Compose(trans)

In [20]:
import torch
import pandas as pd
import torch.utils.data as Data
import torch.nn.functional as F

batch_sizes = 16

df = pd.read_csv('fashion-mnist_train.csv')

x_train = df.iloc[:1000,1:]
y_train = df.iloc[:1000,:1]


x_train_tensor = torch.FloatTensor(x_train.values)
y_train_tensor = torch.FloatTensor(y_train.values)

train_dataset = Data.TensorDataset(x_train_tensor, y_train_tensor)

train_dataloader = Data.DataLoader(train_dataset,batch_sizes,shuffle=True,drop_last =True)
len(train_dataloader)
print(df)

       label  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0          2       0       0       0       0       0       0       0       0   
1          9       0       0       0       0       0       0       0       0   
2          6       0       0       0       0       0       0       0       5   
3          0       0       0       0       1       2       0       0       0   
4          3       0       0       0       0       0       0       0       0   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
59995      9       0       0       0       0       0       0       0       0   
59996      1       0       0       0       0       0       0       0       0   
59997      8       0       0       0       0       0       0       0       0   
59998      8       0       0       0       0       0       0       0       0   
59999      7       0       0       0       0       0       0       0       0   

       pixel9  ...  pixel775  pixel776 

In [21]:
test_df = df.sample(frac=0.2)

x_test = df.iloc[:200,1:]
y_test = df.iloc[:200,:1]


x_test_tensor = torch.FloatTensor(x_test.values)
y_test_tensor = torch.FloatTensor(y_test.values)

test_dataset = Data.TensorDataset(x_test_tensor, y_test_tensor)

test_dataloader = Data.DataLoader(test_dataset,batch_sizes,shuffle=True,drop_last =True)



In [22]:
import torch.nn as nn
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss = nn.CrossEntropyLoss()
model = VGG11(1,10).to(device)
optimizer = optim.Adam(model.parameters(),lr = 1e-3)


In [26]:
# 训练模型


def train_model(dataloader):
    model.train()
    
    total_loss = 0
    for idx,(x,y) in enumerate(dataloader):
        y = y.long()
        y = torch.squeeze(y)
        x = x.to(device)
        y = y.to(device)
        y_pred = model(x)
        cur_loss = loss(y_pred,y)
        optimizer.zero_grad()
        cur_loss.backward()
        optimizer.step()
        
        total_loss += cur_loss.item()
    print(f"train loss:{total_loss/len(train_dataset)}")
    model.eval()
    return total_loss/len(train_dataset)

In [27]:
def test_model(dataloader):
    model.eval()
    
    total_loss = 0
    for idx,(x,y) in enumerate(dataloader):
        y = y.long()
        y = torch.squeeze(y)
        x = x.to(device)
        y = y.to(device)
        y_pred = model(x)
        cur_loss = loss(y_pred,y)
        
        total_loss += cur_loss.item()
    print(f"train loss:{total_loss/len(train_dataset)}")
    return total_loss/len(train_dataset)

In [ ]:
import matplotlib.pyplot as plt

num_epoch = 10
train_losses = []
test_losses = []

for i in range(num_epoch):
    print(f"==== Epoch {i+1} ====")
    train_loss = train_model(train_dataloader)
    test_loss = test_model(test_dataloader)
    train_losses.append(train_loss)
    test_losses.append(test_loss)



plt.plot(range(1, num_epoch + 1), train_losses, label='Training Loss')
plt.plot(range(1, num_epoch + 1), test_losses, label='Test Loss')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Test Loss over Epochs')
plt.legend()

plt.show

==== Epoch 1 ====
train loss:0.14690384006500243
train loss:0.0286588077545166
==== Epoch 2 ====
train loss:0.14696634006500245
train loss:0.0285963077545166
==== Epoch 3 ====
train loss:0.14696634006500245
train loss:0.0286588077545166
==== Epoch 4 ====
train loss:0.14696634006500245
train loss:0.0285963077545166
==== Epoch 5 ====
train loss:0.14709134006500243
train loss:0.0285963077545166
==== Epoch 6 ====
train loss:0.14715384006500243
train loss:0.0285963077545166
==== Epoch 7 ====
train loss:0.14709134006500243
train loss:0.0286588077545166
==== Epoch 8 ====
train loss:0.14702884006500244
train loss:0.0285963077545166
==== Epoch 9 ====
train loss:0.14702884006500244
train loss:0.0285963077545166
==== Epoch 10 ====
train loss:0.14715384006500243
train loss:0.0285963077545166
